# Polymers

  __Goal__ : Study of Polymers configurations at the thermodynamical equilibrium, with a simulation via Metropolis Algorithm, dependig on the interaction parameter

1) Load Libraries and the Experiment file

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from mpl_toolkits.mplot3d import Axes3D
import random

%matplotlib qt 

#!pip freeze > requirements.txt
plt.close('all') #close all figures if open


2) Giration Radius Definition

In [2]:
def g_radius(chain):
    
    d=chain.shape[1]  #i'm taking columns n,m,l (row, columns, z) like how many cols
    r_cm=np.mean(chain,0)
    r_g=0;
    
    for i in range(chain.shape[0]): #iteration along rows
        for j in range(d):          #iter along cols --> analysis of the matrix
            r_g=r_g+(chain[i,j-1]-r_cm[j-1])**2
    
    r_g=np.sqrt(r_g/chain.shape[0])
    #return r_g, r_cm;
    return r_g;

In [34]:
#The goal is to move each monomer along the chain using the Metropolis algorithm and to determine if the new position is accepted based on the Metropolis criteria.
np.random.randint(1,[2,10,6])

array([1, 5, 4])

## Metropolis Simulation Step

3) Metropolis Algorithm for the "Diffusion" of the Self-Avoiding Chain, (single step of the simulation)

In [3]:
def Metropolis_diff(chain, interact): # motion of the monomers along the chain
                                        #chain: A 2D numpy array representing the positions of monomers along the chain.
                                        #interact: A parameter that controls the interaction energy between monomers.

    d=chain.shape[1]
    L=chain.shape[0]
    
    #1° monomer
    dice_diff= -1 +np.random.randint(0,3,d); #0<n<3 with d= n° of elements
    new_pos=chain[0,:]+dice_diff;    #new position of the 1° monomer 
    
    acc=-100;
    acc=np.sum(np.prod((chain[:]==new_pos),1)) #evaluate if the new position is already occupied in the chain
    
    r_post=new_pos-chain[1,:]        #distance with of the 2° monomer
    r2_post=0;
    for j in range(d):
        r2_post=r2_post+r_post[j]**2    #square distance
        
    if (r2_post==1)&(acc==0): #positional check : new position is valid with no overlapp
        new_chain=np.copy(chain);
        new_chain[0,:]=chain[0,:]+dice_diff;
        Delta_NN=NN_count(new_chain,1)-NN_count(chain,1)  #energetic evaluation: E change
        boltz=np.exp(-interact*Delta_NN);   #e^beta*dE
        dice_acc=np.random.random() 
        if dice_acc<np.min([1,boltz]):   #transition probability
            chain[0,:]=new_pos;   #accept the new position of the monomer
  
    #move monomers from 2 to L-1
    for i in range(1,L-1):
        #generate radom movement for each monomers
        dice_diff=-1+np.random.randint(0,3,d);
        new_pos=chain[i,:]+dice_diff;           #new position of the i° monomer
        
        r_pre=new_pos-chain[i-1,:]              #distance with of the i-1° monomer
        r_post=new_pos-chain[i+1,:]             #distance with of the i+1° monomer
        r2_pre=0;
        r2_post=0;
        
        for j in range(d):  #compute distance along 3D
            r2_pre=r2_pre+r_pre[j]**2
            r2_post=r2_post+r_post[j]**2

                
        acc=-100;
        acc=np.sum(np.prod((chain[:]==new_pos),1)); #evaluate if the new position is already occupied in the chain
        
        if (r2_pre==1)&(r2_post==1)&(acc==0):  #positional check ù

            #energetic eveluation and transition probs
            new_chain=np.copy(chain);
            new_chain[i,:]=new_chain[i,:]+dice_diff;
            Delta_NN=NN_count(new_chain,i)-NN_count(chain,i) #energetic evaluation
            boltz=np.exp(-interact*Delta_NN)
            dice_acc=np.random.random()
            if dice_acc<np.min([1,boltz]):   #transition probability
                chain[i,:]=new_pos;     #accept the new position of the monomer
    
    #Move the last monomer
    dice_diff=-1+np.random.randint(0,3,d);
    new_pos=chain[L-1,:]+dice_diff;
    r_pre=new_pos-chain[L-2,:]
    r2_pre=0;
    acc=-100;
    acc=np.sum(np.prod((chain[:]==new_pos),1))
    

    for j in range(d):
        r2_pre=r2_pre+r_pre[j]**2
    if (r2_pre==1)&(acc==0): #like if they overlpa, chang etheir positions
        new_chain=np.copy(chain)
        new_chain[L-1,:]=chain[L-1,:]+dice_diff
        Delta_NN=NN_count(new_chain,L-1)-NN_count(chain,L-1)
        boltz=np.exp(-interact*Delta_NN)
        dice_acc=np.random.random()
        if dice_acc<np.min([1,boltz]):
            chain[L-1,:]=new_pos;
    

    #FINAL COUNT OF NEAREST NEIGHBORS for each monomers
    nn_fin=0;
    for i in range(L):
        nn_fin=nn_fin+NN_count(chain,i)
    
    return(chain, nn_fin)

In [37]:
print (np.random.randint(0,3,3)-1, np.random.randint(0,3,3))

[1 1 0] [2 0 1]


4) Define an Energy of the Chain

In [70]:
def NN_count(chain, ind): 
    d=chain.shape[1]
    L=chain.shape[0]
    nn=0
    
    for i in range(d):
        direction=np.zeros(d);
        direction[i-1]=direction[i-1]+1
        nn=nn+np.sum(np.prod((chain[:]==(chain[ind]+direction)),1)); #counting the NN in the nearest sites
        nn=nn+np.sum(np.prod((chain[:]==(chain[ind]-direction)),1));

    return(nn)  #tot n° of nearest neighbors for monomers at a given index


def NN_tot (chain):
    for i in L:
        NN_tot += NN_count(chain, i)
    return NN_tot

5) Choice of the Initial Chain for the Simulation

In [11]:
d=3
L=20
lin_chain=np.zeros((L,d)) #create the chain: nitialize a 2D array of zeros with dimensions L x d.
for i in range(L):
    lin_chain[i,-1]=i  #set initial position

d=(len(lin_chain.transpose()))

fig = plt.figure(6);

if d==2:
    ax=fig.add_subplot();
    line1,= ax.plot(lin_chain[:,0],lin_chain[:,1],'-o')
    ax.axis('equal')
else:
    ax=fig.add_subplot(projection='3d');
    line1,= ax.plot(lin_chain[:,0],lin_chain[:,1],lin_chain[:,2],'-o')

    
plt.ion()  #Enable interactive mode for dynamic updating.

initial_chain=lin_chain

6) Statistical Trajectory of a Chain, with interactions

In [209]:
chain=np.copy(initial_chain); 

interact = +0.8;   # interact>0 => repulsive, interact<0 => attractive   #EPSILOM

d=chain.shape[1]
L=chain.shape[0]

steps=5000; #step of the simulation
    
plot_freq=round(steps/100); # plot every plot_freq steps

fig = plt.figure(70);

if d==2:
    ax=fig.add_subplot();
    line1,= ax.plot(chain[:,0],chain[:,1],'-o')
    ax.axis('equal')
else:
    ax=fig.add_subplot(projection='3d');
    line1,= ax.plot(chain[:,0],chain[:,1],chain[:,2],'-o')
    
plt.ion()


In [216]:
interact = 0
R_g_results= []
for i in range(steps): # simulation  
        chain, NN_tot = Metropolis_diff(chain, interact)  #uses NN count funct
       
        #NN_tot_results.append(NN_tot)

            
        if np.mod(i,plot_freq)==0:  #resto = np.mod
            #print(100*i/steps, '%')
            line1.set_xdata(chain[:,0]);
            line1.set_ydata(chain[:,1]);
            
            
            ax.axis([np.min(chain[:,0])-0.15*np.sqrt(L),
                    np.max(chain[:,0])+0.15*np.sqrt(L),
                    np.min(chain[:,1])-0.15*np.sqrt(L),
                    np.max(chain[:,1])+0.15*np.sqrt(L),
                    np.min(chain[:,2])-0.15*np.sqrt(L),
                    np.max(chain[:,2])+0.15*np.sqrt(L)])
            
            if d==3:
                line1.set_3d_properties(chain[:,2])
        

            plt.title(f" $\epsilon$: {interact} repulsive")

            plt.show()
            plt.pause(0.02)
            plt.tight_layout()
print (g_radius(chain))


1.8986837546047528


7) Plot significant quantities vs steps


In [ ]:

#This complete workflow initializes a chain of monomers, sets up a Metropolis simulation with interaction energies, and dynamically visualizes the chain's evolution over time. Each part of the code serves to initialize the system, perform the Metropolis steps, and visualize the results, ensuring that the chain configuration is updated and displayed correctly throughout the simulation.

8) Statistics of the Trajectory: histograms? mean values and standard deviations?

In [144]:
epsilom = np.linspace(-1,1,10)
NN_tot_result= [54, 48, 48, 40, 40, 46, 40, 46, 46, 38]
#[48, 54, 44, 44, 40, 40, 42, 38, 42, 40] #[58, 52, 46, 38, 44, 44, 38, 44, 40, 40]
plt.plot(epsilom, NN_tot_result, linestyle = ":")
plt.xlabel("$\epsilon$")
plt.ylabel("tot Nearest Neighbors")
plt.title("NN_tot ($\epsilon$)")

Text(0.5, 1.0, 'NN_tot ($\\epsilon$)')

## Assignments:

3) At the equilibrium, study R_g and NN_tot  of a linear chain with L=20 in 3D, as a function of the interaction parameter 𝜀, and comment the results.

4) Define a new potential (simple but significant for a polymer) for the chain, and study the equilibrium properties (Optional)


In [143]:
R_g_results = []
NN_tot_result= []

epsilons = np.linspace(-1, 1, 10)
for epsilon in epsilons:

    chain=np.copy(initial_chain); 

    #interact= +0.8;   # interact>0 => repulsive, interact<0 => attractive   #EPSILOM

    d=chain.shape[1]
    L=chain.shape[0]

    steps=5000;

    for i in range(steps): # simulation  
        chain, NN_tot = Metropolis_diff(chain, epsilon)  #uses NN count funct
        
    NN_tot_result.append(NN_tot)
print (NN_tot_result)


[54, 48, 48, 40, 40, 46, 40, 46, 46, 38]


In [307]:
#energy of the chain at each step

import plotly.graph_objects as go

def calculate_energy(epsilon, minimal_distance, distance):
        energy = epsilon * (((minimal_distance / distance) ** 12) - (2 * ((minimal_distance / distance) ** 6)))
        return energy

epsilons =[-0.9, -0.2,-0.15, 0.1, 0.15, 0.2, 0.9]
distances= np.arange(0.5, 3, 0.1)
minimal_distance = 2.1 #R_g a cui e=o


for epsilon in epsilons:
        Ue= []

        for distance in distances :
                Ue.append(calculate_energy(epsilon, minimal_distance, distance))
                

                

        plt.plot((distances), (Ue), marker= "o", markersize= 2.8, linestyle= "--", label= f" $\epsilon$ {epsilon}" )
        plt.xlabel("distance")
        plt.ylabel("Potential energy")
        plt.title( "Potential Energy (d)")
        plt.legend()
        plt.grid(True)
        plt.show()

#0.1: purple, 0.6 lightblue, 

In [257]:
epsilons =[ -0.2, 0.1, 0.2]
Ue= []

for epsilon in epsilons:

        distances= np.arange(0.4, 2, 0.1)
        minimal_distance = 1.98 #R_g a cui e=o

        for distance in distances :

                def calculate_energy(epsilon, minimal_distance, distance):
                    energy = epsilon * (((minimal_distance / distance) ** 12) - (2 * ((minimal_distance / distance) ** 6)))
                    return energy



                potential_energy_scan = [calculate_energy(epsilon, minimal_distance, distance) for distance in distances]
                Ue.append(potential_energy_scan)



        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=list(distances), y=potential_energy_scan,
                mode='lines+markers',
                name='transaction-based model',
                marker=dict(size=10)
            )
        )

        fig.update_layout(legend=dict(itemsizing='constant'))
        fig.update_layout(legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1.05,
                font = dict(family = "Arial", size = 20),
                bordercolor="LightSteelBlue",
                borderwidth=2,
            ),
        )
        fig.update_traces(line=dict(width=5))

        fig.update_layout(
            template='simple_white',
            xaxis_tickformat = 'i',
            bargap=0.2, # gap between bars of adjacent location coordinates,
            height=600,
            width=1350
        )


fig.show()